In [38]:
import glob
import sys
import os
import json
import pickle
import logging
from qdrant_client import QdrantClient, models
from sentence_transformers import SentenceTransformer
from dotenv import find_dotenv, load_dotenv
import json
from sentence_transformers import CrossEncoder
from helpers import chat
from octoai.client import Client
from helpers import chat
import ast

# root_path = '/Users/sshrivastava/PycharmProjects/entertainment-hub-main'
root_path = '/home/ec2-user/sarang/wiki_cheat'

sys.path.insert(0, os.path.abspath(root_path))
os.chdir(root_path)

import openai
### Load the environment file
env_file_path = find_dotenv()
logging.info(f'env_file_path: {env_file_path}')
load_dotenv(env_file_path)

### API keys and tokens needed for interaction with external APIS
openai.api_key = os.getenv('OPENAI_API_KEY')
huggingface_token = os.getenv('HF_TOKEN')

# Establish connection to Pinecone server

In [5]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key="8777d46f-ba38-4551-a016-baa6a6952549", pool_threads=30)

# Load the test data

In [13]:
test_data = None
test_data_path = 'data/test_data.json'
with open(test_data_path, 'r') as fp:
    test_data = json.load(fp)

In [14]:
len(test_data), all_data[0]

(2000,
 {'dp': {'id': 1,
   'title': '24-hour clock',
   'text': 'A time in the 24-hour clock is written in the form hours:minutes (for example, 01:23), or hours:minutes:seconds (01:23:45). Numbers under 10 have a zero in front (called a leading zero); e.g. 09:07. Under the 24-hour clock system, the day begins at midnight, 00:00, and the last minute of the day begins at 23:59 and ends at 24:00, which is identical to 00:00 of the following day. 12:00 can only be mid-day. Midnight is called 24:00 and is used to mean the end of the day and 00:00 is used to mean the beginning of the day. For example, you would say "Tuesday at 24:00" and "Wednesday at 00:00" to mean exactly the same time.',
   'url': 'https://simple.wikipedia.org/wiki?curid=9985',
   'wiki_id': 9985,
   'views': 2450.62548828125,
   'paragraph_id': 1,
   'langs': 30},
  'qa': {'question': 'What is the format for representing time in the 24-hour clock system?',
   'answer': 'A time in the 24-hour clock is written in the form

# V1.0 Baseline Solution: Encode each paragraph of the wikipedia articles using all_mpnet_base_v2. Not using any reranker here. 

### https://api.wandb.ai/links/sarang24s/iwj6h7di



In [4]:
encoder_model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2', device='cuda')

/home/ec2-user/anaconda3/envs/rag2/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [26]:
index = pc.Index("wiki-all-mpnet-base-v2")

In [10]:
results = []
vectors = encoder_model.encode([dp['qa']['question'] for dp in test_data], show_progress_bar=True, batch_size=512).tolist()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [11]:
%%time
for idx, dp in enumerate(test_data):
    print(idx)
    result = index.query(
        vector=vectors[idx],
        top_k=100,
        include_metadata=True
    )
    results.append(result['matches'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [13]:
top_ks = [1,2,3,5,7,10,20,30,50,75,100]
recall_topk = {}

for top_k in top_ks:

    correct_dp = 0
    
    for dp, result in zip(test_data,results):
    
        for res in result[:top_k]:
            if int(res['metadata']['id']) == dp['dp']['id']:
                correct_dp += 1

    print(f"top_k{top_k} recall {round(correct_dp/len(test_data),2)}")
    recall_topk[top_k] = round(correct_dp/len(test_data),2)
                
    

top_k1 recall 0.7
top_k2 recall 0.82
top_k3 recall 0.88
top_k5 recall 0.91
top_k7 recall 0.93
top_k10 recall 0.94
top_k20 recall 0.96
top_k30 recall 0.97
top_k50 recall 0.97
top_k75 recall 0.98
top_k100 recall 0.98


In [47]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="wiki_cheat",
    name = "baseline_run",
    notes = "This was a baseline run. We are only using all_mpnet_base_v2 as the base model",
    # track hyperparameters and run metadata
    config={
        "retriever_model": 'sentence-transformers/all-mpnet-base-v2',
        "reranker_model": "None",
        "chunking": "Default/paragraph level in the dataset",
        "vector_db": "Default",
    }
)

for key, val in recall_topk.items():
    wandb.log({"top_k": key, "recall": val})

wandb.finish()

recall,▁▄▅▆▇▇▇████
top_k,▁▁▁▁▁▂▂▃▄▆█
recall,0.98
top_k,100


# V1.1: Use a reranker on top of retrieved results. Use off the shelf re ranker.

In [16]:
results[0]

[{'id': '0',
  'metadata': {'id': 0.0,
               'langs': 30.0,
               'paragraph_id': 0.0,
               'text': 'The 24-hour clock is a way of telling the time in which '
                       'the day runs from midnight to midnight and is divided '
                       'into 24 hours, numbered from 0 to 23. It does not use '
                       'a.m. or p.m. This system is also referred to (only in '
                       'the US and the English speaking parts of Canada) as '
                       'military time or (only in the United Kingdom and now '
                       'very rarely) as continental time. In some parts of the '
                       'world, it is called railway time. Also, the '
                       'international standard notation of time (ISO 8601) is '
                       'based on this format.',
               'title': '24-hour clock',
               'url': 'https://simple.wikipedia.org/wiki?curid=9985',
               'views': 24

In [14]:
from sentence_transformers import CrossEncoder
model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-12-v2', max_length=512, device='cuda')


In [17]:
reranked_results = []
cnt = 0
for dp, result in zip(test_data, results):
    predictions = [ (dp['qa']['question'], res['metadata']['text']) for res in result]
    scores = model.predict(predictions).tolist()

    results_with_scores = [ {'res':res, 'score':scores[idx]} for idx, res in enumerate(result)]
    reranked_result = []
    for result in sorted(results_with_scores, key=lambda x: x['score'], reverse=True):
        result['res']['rerank_score'] = result['score']
        reranked_result.append(result['res'])
    
    reranked_results.append(reranked_result)
    cnt += 1
    print(cnt)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [18]:
top_ks = [1,2,3,5,7,10,20,30,50,75,100]
recall_topk = {}

for top_k in top_ks:

    correct_dp = 0
    
    for dp, result in zip(test_data,reranked_results):
    
        for res in result[:top_k]:
            if int(res['metadata']['id']) == dp['dp']['id']:
                correct_dp += 1

    print(f"top_k{top_k} recall {round(correct_dp/len(test_data),2)}")
    recall_topk[top_k] = round(correct_dp/len(test_data),2)
                
    

top_k1 recall 0.87
top_k2 recall 0.93
top_k3 recall 0.95
top_k5 recall 0.96
top_k7 recall 0.97
top_k10 recall 0.97
top_k20 recall 0.97
top_k30 recall 0.97
top_k50 recall 0.98
top_k75 recall 0.98
top_k100 recall 0.98


In [89]:
import wandb
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="wiki_cheat",
    name = "v1.0_run",
    notes = "We used all_mpnet_base_v2 as the base encoder model and ms-marco-MiniLM-L-12-v2 as the reranking model",
    # track hyperparameters and run metadata
    config={
        "retriever_model": 'sentence-transformers/all-mpnet-base-v2',
        "reranker_model": "ms-marco-MiniLM-L-12-v2",
        "chunking": "Default/paragraph level in the dataset",
        "vector_db": "Default",
    }
)

for key, val in recall_topk.items():
    wandb.log({"top_k": key, "recall": val})

wandb.finish()

recall,▁▅▆▇▇▇▇▇███
top_k,▁▁▁▁▁▂▂▃▄▆█
recall,0.98
top_k,100


# v2.0 Use a trained embedder model

In [16]:
model_path = 'train_embedder/models/sentence-transformers-all-mpnet-base-v2-2024-01-27_20-14-10'
index_name = "wiki-all-mpnet-base-v2-trained"
encoder_model = SentenceTransformer(model_path, device='cuda')
index = pc.Index(index_name)
vectors = encoder_model.encode([dp['qa']['question'] for dp in test_data], show_progress_bar=True, batch_size=512).tolist()

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
%%time
results = []
for idx, dp in enumerate(test_data):
    print(idx)
    result = index.query(
        vector=vectors[idx],
        top_k=100,
        include_metadata=True
    )
    results.append(result['matches'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [19]:
top_ks = [1,2,3,5,7,10,20,30,50,75,100]
recall_topk = {}
for top_k in top_ks:
    correct_dp = 0
    for dp, result in zip(test_data,results):
        for res in result[:top_k]:
            if int(res['metadata']['id']) == dp['dp']['id']:
                correct_dp += 1
    print(f"top_k{top_k} recall {round(correct_dp/len(test_data),2)}")
    recall_topk[top_k] = round(correct_dp/len(test_data),2)

top_k1 recall 0.8
top_k2 recall 0.89
top_k3 recall 0.93
top_k5 recall 0.95
top_k7 recall 0.96
top_k10 recall 0.97
top_k20 recall 0.98
top_k30 recall 0.98
top_k50 recall 0.99
top_k75 recall 0.99
top_k100 recall 0.99


In [20]:
import wandb
import random

wandb.init(
    # set the wandb project where this run will be logged
    project="wiki_cheat",
    name = "v2.0",
    notes = "This run was made using a trained embedder model",
    # track hyperparameters and run metadata
    config={
        "retriever_model": model_path,
        "reranker_model": "None",
        "chunking": "Default/paragraph level in the dataset",
        "vector_db": "Default",
    }
)

for key, val in recall_topk.items():
    wandb.log({"top_k": key, "recall": val})

wandb.finish()

wandb: Currently logged in as: sarang24s. Use `wandb login --relogin` to force relogin


recall,▁▄▆▇▇▇█████
top_k,▁▁▁▁▁▂▂▃▄▆█
recall,0.99
top_k,100


# v2.1 Use a trained reranker model on top of retrieved results

In [23]:
reranked_model_path = 'train_reranker/models/cross-encoder-ms-marco-MiniLM-L-12-v2-2024-01-28_19-22-20'
model = CrossEncoder(reranked_model_path, max_length=512, device='cuda')

In [24]:
%%time
reranked_results = []
cnt = 0
for dp, result in zip(test_data, results):
    predictions = [ (dp['qa']['question'], res['metadata']['text']) for res in result]
    scores = model.predict(predictions).tolist()

    results_with_scores = [ {'res':res, 'score':scores[idx]} for idx, res in enumerate(result)]
    reranked_result = []
    for result in sorted(results_with_scores, key=lambda x: x['score'], reverse=True):
        result['res']['rerank_score'] = result['score']
        reranked_result.append(result['res'])
    
    reranked_results.append(reranked_result)
    cnt += 1
    print(cnt)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [25]:
top_ks = [1,2,3,5,7,10,20,30,50,75,100]
recall_topk = {}

for top_k in top_ks:

    correct_dp = 0
    
    for dp, result in zip(test_data,reranked_results):
    
        for res in result[:top_k]:
            if int(res['metadata']['id']) == dp['dp']['id']:
                correct_dp += 1

    print(f"top_k{top_k} recall {round(correct_dp/len(test_data),2)}")
    recall_topk[top_k] = round(correct_dp/len(test_data),2)
                
    

top_k1 recall 0.86
top_k2 recall 0.92
top_k3 recall 0.94
top_k5 recall 0.95
top_k7 recall 0.96
top_k10 recall 0.97
top_k20 recall 0.98
top_k30 recall 0.99
top_k50 recall 0.99
top_k75 recall 0.99
top_k100 recall 0.99


In [27]:
import wandb
import random

wandb.init(
    # set the wandb project where this run will be logged
    project="wiki_cheat",
    name = "v2.1",
    notes = "This run was made using a trained embedder model and runnign a trained reranked model on top of it",
    # track hyperparameters and run metadata
    config={
        "retriever_model": model_path,
        "reranker_model": reranked_model_path,
        "chunking": "Default/paragraph level in the dataset",
        "vector_db": "Default",
    }
)

for key, val in recall_topk.items():
    wandb.log({"top_k": key, "recall": val})

wandb.finish()

recall,▁▄▅▆▆▇▇████
top_k,▁▁▁▁▁▂▂▃▄▆█
recall,0.99
top_k,100


# V2.2 Using the original reranker on top of retrieved results

In [26]:
reranked_model_path = 'cross-encoder/ms-marco-MiniLM-L-12-v2'
model = CrossEncoder(reranked_model_path, max_length=512, device='cuda')

/home/ec2-user/anaconda3/envs/rag2/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [28]:
%%time
reranked_results = []
cnt = 0
for dp, result in zip(test_data, results):
    predictions = [ (dp['qa']['question'], res['metadata']['text']) for res in result]
    scores = model.predict(predictions).tolist()

    results_with_scores = [ {'res':res, 'score':scores[idx]} for idx, res in enumerate(result)]
    reranked_result = []
    for result in sorted(results_with_scores, key=lambda x: x['score'], reverse=True):
        result['res']['rerank_score'] = result['score']
        reranked_result.append(result['res'])
    
    reranked_results.append(reranked_result)
    cnt += 1
    print(cnt)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [81]:
top_ks = [1,2,3,5,7,10,20,30,50,75,100]
recall_topk = {}

for top_k in top_ks:

    correct_dp = 0
    
    for dp, result in zip(test_data,reranked_results):
    
        for res in result[:top_k]:
            if int(res['metadata']['id']) == dp['dp']['id']:
                correct_dp += 1

    print(f"top_k{top_k} recall {round(correct_dp/len(test_data),2)}")
    recall_topk[top_k] = round(correct_dp/len(test_data),2)
                
    

top_k1 recall 0.88
top_k2 recall 0.94
top_k3 recall 0.96
top_k5 recall 0.97
top_k7 recall 0.98
top_k10 recall 0.98
top_k20 recall 0.98
top_k30 recall 0.99
top_k50 recall 0.99
top_k75 recall 0.99
top_k100 recall 0.99


In [30]:
import wandb
import random

wandb.init(
    # set the wandb project where this run will be logged
    project="wiki_cheat",
    name = "v2.2",
    notes = "This run was made using a trained embedder model and running the baseline rerank model on top of it",
    # track hyperparameters and run metadata
    config={
        "retriever_model": model_path,
        "reranker_model": reranked_model_path,
        "chunking": "Default/paragraph level in the dataset",
        "vector_db": "Default",
    }
)

for key, val in recall_topk.items():
    wandb.log({"top_k": key, "recall": val})

wandb.finish()

recall,▁▅▆▇▇▇▇████
top_k,▁▁▁▁▁▂▂▃▄▆█
recall,0.99
top_k,100


# v2.3 Using a LLM reader on top of top-10 results to improve the results further

In [35]:
def get_reader_prompt_v2():
    """
    This function returns a multiline string that serves as a prompt for reading the reranked data
    """
    prompt = """You are an asssistant to a Machine Learning Engineer. We are developing a Question Answering assistant that will work on top of 
    wikipedia articles. For an input question, our system(reader/retriever pipeline) returns a set of relevant wikipedia passages. You need to 
    find right passage that contains the answer to the question asked. 
    --- INPUT 1: Input question along with a set of relevant passages for it. Each passage has an id associated with it. This input is delimited by triple @.
    @@@{data_for_prompt}@@@
    --- TASK 1: You need to find the id of the passage that contains the answer to the question asked, provide a step by step 
    reasoning for your answer and find the answer to the question from the passage.
    --- CONSTRAINT 1: You should return a dictionary. That dictionary should have 3 keys. 
    First key is the 'id' which is the id of the passage that contains the answer. Second is the 'reason' which represents a step by step 
    reasoning for your answer. And third is 'answer' which is a small part of the input passage that contains the answer.
    --- CONSTRAINT 2: If the answer is not present in any of the input passages you should return "-1" as the id, reason should be "passages don't contain the answe" and 'evidence' should be "None"
    --- CONSTRAINT 3: You should not add any extra text before/after the json object that you return. Just return a list of dictionaries without any text before or after.
    """
    return prompt

In [86]:
def get_reader_prompt_v3():
    """
    This function returns a multiline string that serves as a prompt for reading the reranked data
    """
    prompt = """You are working as a Question Answering assistant. You will be given a question along with 10 passages. You need to find the id 
    of the passage that contains the answer to the question asked, answer the question and provide a step by step reasoning for your answer.
    --- INPUT 1: Input Question. This input is delimited by triple @.
    @@@{input_question}@@@
    --- INPUT 2: Set of passages. This input is delimited by triple @.
    @@@{input_passages}@@@
    --- TASK 1: You need to find the id of the passage that contains the answer to the question asked, 
    answer the question and provide a step by step reasoning for your answer.
    --- CONSTRAINT 1: You should return a dictionary. That dictionary should have 3 keys. 
    First key is the 'id' which is the id of the passage that contains the answer, second key is 'answer' which is the answer to the question, 
    and Thrid key is the 'reason' which represents a step by step 
    reasoning for your answer. 
    --- CONSTRAINT 2: If the answer is not present in any of the input passages you should return a dictionary with id as -1, answer as None and reason as Not present.
    --- CONSTRAINT 3: You should not add any extra text before/after the json object that you return. Just return a list of dictionaries without any text before or after.
    """
    return prompt

## Output from Mistral 7B Reader
### 1) It keeps adding extra pieces of information between answers. 
### 2) It does not limits its answer to a single best candidate answer. Can return multiple answers even when explicitly being asked in the input prompt
### 3) It is not able to understand if the retrieved passages don't have an answer in them.

In [ ]:
client = Client()

cnt = 0
for dp, reranked_result in zip(test_data[96:100], reranked_results[96:100]):

    question = dp['qa']['question']
    relevant_passages = [ { 'passage': res['metadata']['text'], 'id': idx } for idx, res in enumerate(reranked_result) ]
    
    prompt=  get_reader_prompt_v2()
    prompt = prompt.format(**{"data_for_prompt": {'question':question, 'relevant_passages':relevant_passages[:10]}} )

    ## Call Mistral 7B
    completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt
            }
        ],
        model="mistral-7b-instruct-fp16",
        max_tokens=512,
        presence_penalty=0,
        temperature=0.001,
        top_p=0.9,
    )

    print(completion.choices[0].message.content)
    answer = ast.literal_eval(completion.choices[0].message.content)
    answers.append(answer)
    cnt+=1
    if cnt > 100:
        break
    print(cnt)

In [33]:
[{"id": 0, "reason": "The passage describes the format for representing time in the 24-hour clock system, which matches the input question.", "answer": "The 24-hour clock is a way of telling the time in which the day runs from midnight to midnight and is divided into 24 hours, numbered from 0 to 23."}]

If the answer is not present in any of the input passages:
[-1, "passages don't contain the answer", None]
1
 [{'id': 0, 'reason': 'The passage mentions that the Catholic Church uses the Vulgate translation of the Old Testament and the Eastern Orthodox church uses the Septuagint. The question asks for the translations used by these churches, so this passage is relevant.', 'answer': 'The Catholic Church uses Saint Jerome\'s Latin translation of the Old Testament called Vulgate. The Eastern Orthodox church uses the ancient Greek translation of Jewish sacred writings called the Septuagint.'}]
2
 [{"id": 0, "reason": "The passage mentions that in 'The Dark Knight Rises', Gotham City was shot in Pittsburgh, Pennsylvania, New York City, and Los Angeles, California. The question asks for the cities where Gotham City was filmed for 'The Dark Knight Rises'.", "answer": "Gotham City was shot in Pittsburgh, Pennsylvania, New York City, and Los Angeles, California."}]
3
 [{"id": 0, "reason": "The passage mentions Abella Danger winning the National Award for the Defense of Human Rights in Colombia in 2021.", "answer": "In 2021, she was the winner of the National Award for the Defense of Human Rights in Colombia."},
 {"id": 1, "reason": "The passage mentions Abella Danger winning Best Pornographic actor in 2021, but this is not the answer to the question.", "answer": "Amongst many awards, she won Best Pornographic actor in 2021 (Pornovizija '21), nominated by the competent jury committee."},
 {"id": 3, "reason": "The passage mentions Abella Danger winning the 'Fierce Women' award for her activism in 2021.", "answer": "In 2021 she won the 'Fierce Women' award for her activism."},
 {"id": -1, "reason": "passages don't contain the answer", "answer": "None"}]

The question asks for the award Abella Danger won in 2021, and the passages mention her winning the National Award for the Defense of Human Rights in Colombia, Best Pornographic actor award, and the 'Fierce Women' award. The first and third passages contain the answer to the question, so they are included in the list with the corresponding reasoning. The second passage mentions a different award, so it is not included in the list. The last dictionary is included to handle the case where the answer is not present in any of the input passages.
4
 [{"id": 0, "reason": "The passage describes Cristiano Ronaldo's international career and mentions his first international hat-trick against Northern Ireland on 6 September 2013.", "answer": "His first international hat-trick came in a 4-2 win against Northern Ireland on 6 September 2013."}]

SyntaxError: invalid syntax (1747006614.py, line 3)

## Output from GPT4 reader

###  Baseline topk recall without using a reader

In [75]:
reader_test_data = test_data[:100]
top_k = 1
recall_topk = {}

correct_dp = 0

for dp, result in zip(reader_test_data,reranked_results):
    if int(result[0]['metadata']['id']) == dp['dp']['id']:
        correct_dp += 1

print(f"top_k{top_k} recall {round(correct_dp/len(reader_test_data),2)}")
                
    

top_k1 recall 0.83


### Run 2 with the reader with prompt v3

In [100]:
cnt = 0
for dp, reranked_result in zip(test_data[:100], reranked_results[:100]):

    question = dp['qa']['question']
    relevant_passages = [ {'id': idx, 'text': res['metadata']['text'] } for idx, res in enumerate(reranked_result[:10]) ]
    
    prompt=  get_reader_prompt_v3()
    prompt = prompt.format(**{"input_question":question , "input_passages":relevant_passages})

    print(prompt)
    print("------")
    ## Call GPT4
    try:
        completion = chat(prompt, model="gpt-4-0125-preview",response_format="json_object")
        # completion = chat(prompt, model="gpt-4-1106-preview", response_format="json_object")
    
        print(completion.choices[0].message.content)
        answer = ast.literal_eval(completion.choices[0].message.content)
        answers_2.append(answer)
        
    except:
        print(f"error occurred {cnt}")
        answers_2.append({
            'id':0,
            'answer': "default",
            'reason': "error"
        })
    print("@@@@@@@@@@")
    cnt+=1
    if cnt > 100:
        break
    print(cnt)
    # print(answer)

You are working as a Question Answering assistant. You will be given a question along with 10 passages. 
    You need to find the id of the passage that contains the answer to the question asked, 
    answer the question and provide a step by step reasoning for your answer.
    --- INPUT 1: Input Question. This input is delimited by triple @.
    @@@What did Nelson Mandela describe the social operation as in his speech?@@@
    --- INPUT 2: Set of passages. This input is delimited by triple @.
    @@@[{'id': 0, 'text': "On 18 July 2007 in Johannesburg, South Africa, Nelson Mandela, his wife Graça Machel, and Desmond Tutu convened a group of world leaders to contribute their wisdom, independent leadership and honesty to tackle some of the world's toughest problems. Nelson Mandela announced the formation of this new group, The Elders, in a speech he delivered on his 89th birthday."}, {'id': 1, 'text': '"We stand here today as nothing more than a representative of the millions of our peopl

In [103]:
reader_answers = []
for idx, ans in enumerate(answers_2):
    reader_answers.append([reranked_results[idx][ans['id']]])

In [104]:
reader_test_data = test_data[:100]
top_k = 1
recall_topk = {}

correct_dp = 0

for dp, result in zip(reader_test_data,reader_answers):
    if int(result[0]['metadata']['id']) == dp['dp']['id']:
        correct_dp += 1

print(f"top_k{top_k} recall {round(correct_dp/len(reader_test_data),2)}")
                
    

top_k1 recall 0.8


### Run 3 with the reader with prompt v3 and adding title text to the passage text

In [107]:
cnt = 0
answers_3 = []
for dp, reranked_result in zip(test_data[:100], reranked_results[:100]):

    question = dp['qa']['question']
    relevant_passages = [ {'id': idx, 'text': res['metadata']['title'] +": " + res['metadata']['text'] } for idx, res in enumerate(reranked_result[:10]) ]
    
    prompt=  get_reader_prompt_v3()
    prompt = prompt.format(**{"input_question":question , "input_passages":relevant_passages})

    # print(prompt)
    # print("------")
    ## Call GPT4
    try:
        completion = chat(prompt, model="gpt-4-0125-preview",response_format="json_object")
        # completion = chat(prompt, model="gpt-4-1106-preview", response_format="json_object")
    
        print(completion.choices[0].message.content)
        answer = ast.literal_eval(completion.choices[0].message.content)
        answers_3.append(answer)
        
    except:
        print(f"error occurred {cnt}")
        answers_3.append({
            'id':0,
            'answer': "default",
            'reason': "error"
        })
    # print("@@@@@@@@@@")
    cnt+=1
    if cnt > 100:
        break
    print(cnt)
    # print(answer)

{
  "id": 1,
  "answer": "hours:minutes or hours:minutes:seconds",
  "reason": "The question asks for the format used in the 24-hour clock system to represent time. Passage 1 explicitly describes how time is written in the 24-hour clock system, stating that it is in the form of 'hours:minutes' (e.g., 01:23) or 'hours:minutes:seconds' (e.g., 01:23:45). It also mentions the use of a leading zero for numbers under 10 (e.g., 09:07). This passage directly answers the question by providing the specific formats used in the 24-hour clock system, making it the correct source for the answer."
}
1
{
  "id": 0,
  "answer": "The Catholic Church uses the Vulgate, and the Eastern Orthodox Church uses the Septuagint.",
  "reason": "The passage with id 0 directly addresses the question by stating the translations of the Old Testament used by both the Catholic Church and the Eastern Orthodox Church. It mentions that the Catholic Church uses Saint Jerome's Latin translation of the Old Testament called th

In [109]:
reader_answers = []
for idx, ans in enumerate(answers_3):
    reader_answers.append([reranked_results[idx][ans['id']]])
    
reader_test_data = test_data[:100]
top_k = 1
recall_topk = {}

correct_dp = 0

incorrect_dps = []
for dp, result in zip(reader_test_data,reader_answers):
    if int(result[0]['metadata']['id']) == dp['dp']['id']:
        correct_dp += 1
    else:
        incorrect_dps.append((dp, result))

print(f"top_k{top_k} recall {round(correct_dp/len(reader_test_data),2)}")   

top_k1 recall 0.82


In [111]:
len(incorrect_dps)

18

### Analyze incorrect results from the reader

In [114]:
for data in incorrect_dps[:5]:
    print(data)
    print("########################")

({'dp': {'id': 63, 'title': 'Carles Puigdemont', 'text': 'On 30 October 2017, Puigdemont fled to Belgium in a move to avoid action from the Spanish judiciary. A month later, he was re-elected to the Parliament.', 'url': 'https://simple.wikipedia.org/wiki?curid=540154', 'wiki_id': 540154, 'views': 1542.51953125, 'paragraph_id': 2, 'langs': 81}, 'qa': {'question': 'When did Carles Puigdemont flee to Belgium to avoid action from the Spanish judiciary?', 'answer': 'On 30 October 2017, Puigdemont fled to Belgium'}}, [{'id': '64',
 'metadata': {'id': 64.0,
              'langs': 81.0,
              'paragraph_id': 3.0,
              'text': 'On 2 November, Spanish courts issued an European Arrest '
                      'Warrant against Puigdemont and four other cabinet '
                      'members to Belgian authorities. Two days later, they '
                      'turned themselves in to the Belgian police. Hours '
                      'later, he was released.',
              'title'

### Analyzing mismatches of the reader results

In [110]:
len(incorrect_dps)

18